# Agent 평가 (API 기반)

이 노트북은 **API 기반의 에이전트를 평가**하는 방법을 보여줍니다.

## 핵심 개념

기존 노트북들은 직접 생성한 응답을 평가했지만, 실제 프로덕션 환경에서는:
- **REST API로 배포된 에이전트/챗봇**을 평가해야 합니다
- API 호출을 통해 응답을 받고, 그 응답의 품질을 측정합니다
- 이를 통해 실제 배포 환경에서의 성능을 평가할 수 있습니다

## 평가 시나리오

이 예제에서는:
1. **Azure OpenAI API를 에이전트 API로 가정**합니다
2. 사용자는 자신의 API 서버를 연결하여 동일한 평가를 수행할 수 있습니다
3. `IntentResolutionEvaluator`를 사용하여 에이전트가 사용자의 의도를 제대로 파악하고 응답했는지 평가합니다

## Agent 평가를 위한 Evaluator 소개

Azure AI Evaluation SDK는 에이전트의 다양한 측면을 평가할 수 있는 여러 evaluator를 제공합니다.

### 🎯 에이전트 전용 Evaluators

#### 1. **IntentResolutionEvaluator** (의도 파악 평가)
- **목적**: 에이전트가 사용자의 의도를 올바르게 식별하고 이해했는지 측정
- **평가 기준**: 
  - 사용자의 질문과 에이전트의 응답이 얼마나 일치하는가?
  - 에이전트가 사용자가 원하는 것을 정확히 파악했는가?
- **점수 범위**: 1~5 (5가 가장 우수)
- **출력**: 
  - `intent_resolution`: 점수
  - `intent_resolution_reason`: 평가 이유
  - `intent_resolution_result`: pass/fail (threshold 기준)

#### 2. **TaskAdherenceEvaluator** (작업 준수 평가)
- **목적**: 에이전트의 최종 응답이 할당된 작업을 준수하는지 측정
- **평가 기준**: 
  - 시스템 메시지(프롬프트)에 명시된 역할을 잘 수행했는가?
  - 이전 단계를 고려하여 적절한 응답을 제공했는가?
- **점수 범위**: 1~5 (5가 가장 우수)
- **예시**: "여행 가이드" 역할을 부여받았는데 다른 주제로 답변하면 낮은 점수

#### 3. **ToolCallAccuracyEvaluator** (도구 호출 정확도 평가)
- **목적**: 에이전트가 올바른 함수/도구를 호출했는지 측정
- **평가 기준**:
  - 사용자 요청에 맞는 도구를 선택했는가?
  - 도구에 전달한 파라미터가 정확한가?
  - 불필요한 도구 호출은 없는가?
- **점수 범위**: 1~5 (5가 가장 우수)
- **지원 도구**: File Search, Azure AI Search, Bing Grounding, Code Interpreter 등

### 📊 품질(Quality) Evaluators

#### 4. **ResponseCompletenessEvaluator** (응답 완성도 평가)
- **목적**: 에이전트의 응답이 질문에 대한 완전한 답변을 제공하는지 평가
- **평가 기준**: 답변이 충분한 정보를 포함하고 있는가?
- **점수 범위**: 1~5 (5가 가장 우수)

#### 5. **CoherenceEvaluator** (일관성 평가)
- **목적**: 응답이 논리적으로 일관성 있고 이해하기 쉬운지 평가
- **점수 범위**: 1~5

#### 6. **FluencyEvaluator** (유창성 평가)
- **목적**: 응답이 문법적으로 올바르고 자연스러운지 평가
- **점수 범위**: 1~5

#### 7. **RelevanceEvaluator** (관련성 평가)
- **목적**: 응답이 질문과 직접적으로 관련이 있는지 평가
- **점수 범위**: 1~5

#### 8. **GroundednessEvaluator** (근거 기반성 평가)
- **목적**: 에이전트의 응답이 제공된 컨텍스트/소스에 근거하는지 평가
- **평가 기준**: 환각(hallucination) 없이 사실에 기반한 답변인가?
- **점수 범위**: 1~5

### 🛡️ 안전성(Safety) Evaluators

#### 9. **ContentSafetyEvaluator** (콘텐츠 안전성 평가)
- **목적**: 응답에 유해한 콘텐츠가 포함되어 있는지 확인
- **평가 항목**: 폭력, 성적 콘텐츠, 혐오 표현, 자해 등
- **출력**: 위험 수준 (Very Low, Low, Medium, High)

#### 10. **IndirectAttackEvaluator** (간접 공격 평가)
- **목적**: Jailbreak 공격이나 악의적인 프롬프트 인젝션 감지
- **출력**: 공격 시도 여부

#### 11. **CodeVulnerabilityEvaluator** (코드 취약점 평가)
- **목적**: 생성된 코드에 보안 취약점이 있는지 확인
- **출력**: 취약점 존재 여부

### 📈 평가 점수 해석

모든 evaluator는 공통적으로 다음을 출력합니다:
- **Score**: 1~5 또는 0~1 범위의 점수
- **Reason**: 평가 이유 설명
- **Result**: "pass" 또는 "fail" (threshold 기준)
- **Threshold**: 기본값 또는 사용자 설정 임계값

### 💡 사용 권장사항

1. **에이전트 워크플로우**: IntentResolution, TaskAdherence, ToolCallAccuracy 사용
2. **응답 품질**: ResponseCompleteness, Coherence, Fluency, Relevance 사용
3. **신뢰성**: Groundedness 사용 (RAG 시스템에 필수)
4. **안전성**: ContentSafety, IndirectAttack 사용 (프로덕션 배포 전 필수)

### 🔗 참고 문서
- [Azure AI Foundry - Agent Evaluators](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/agent-evaluate-sdk?view=foundry-classic)

## 1. 필요한 라이브러리 Import

Azure AI Evaluation 라이브러리와 OpenAI 클라이언트를 가져옵니다.

In [1]:
import os
import json
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation import IntentResolutionEvaluator, ResponseCompletenessEvaluator
from openai import AzureOpenAI

## 2. Azure OpenAI 설정

평가자(Evaluator)가 사용할 Azure OpenAI 모델을 설정합니다.
이 설정은 평가 과정에서 응답의 품질을 판단하는데 사용됩니다.

In [ ]:
# Azure OpenAI 엔드포인트 설정
endpoint = "https://<your-aoai-endpoint>.cognitiveservices.azure.com/"
api_key = "<your_key>"
model = "<your_model>"
api_version = "2024-12-01-preview"

# 평가자(Evaluator)를 위한 모델 설정
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    azure_deployment=model,
)

## 3. API 기반 에이전트 함수 정의

이 섹션에서는 **실제 API 에이전트를 시뮬레이션하는 함수**를 만듭니다.

### 중요한 점:
- 실제 프로덕션 환경에서는 이 함수가 **여러분의 REST API 서버를 호출**합니다
- 이 예제에서는 Azure OpenAI API를 에이전트 API로 가정합니다
- 함수는 질문(query)을 받아서 에이전트의 응답을 반환합니다

### 커스터마이징 방법:
아래 `call_agent_api()` 함수의 내부 로직을 수정하여 여러분의 API 서버를 연결할 수 있습니다:
```python
def call_agent_api(query: str) -> str:
    # 여기에 여러분의 API 호출 코드를 작성하세요
    # 예: response = requests.post("https://your-api.com/chat", json={"query": query})
    # return response.json()["answer"]
    pass
```

In [7]:
def call_agent_api(query: str, isTrue: bool) -> str:
    """
    API 기반 에이전트를 호출하는 함수
    
    이 함수는 실제 프로덕션 환경에서 배포된 에이전트 API를 호출하는 것을 시뮬레이션합니다.
    
    Parameters:
    -----------
    query : str
        사용자의 질문
    
    Returns:
    --------
    str
        에이전트의 응답
    
    커스터마이징 가이드:
    ------------------
    이 예제에서는 Azure OpenAI Chat Completion API를 사용하지만,
    여러분의 실제 API 서버로 대체할 수 있습니다:
    
    예시 REST API 호출:
        import requests
        response = requests.post(
            "https://your-api-server.com/chat",
            json={"query": query, "user_id": "test_user"},
            headers={"Authorization": "Bearer YOUR_TOKEN"}
        )
        return response.json()["answer"]

        return response.text
    """
    
    # 현재 예제: Azure OpenAI를 에이전트 API로 가정
    # 실제로는 아래 코드를 여러분의 API 호출 코드로 대체하세요
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        azure_endpoint=endpoint
    )
    
    # 챗봇 시스템 프롬프트 (실제로는 API 서버 측에 정의되어 있을 것입니다)
    # 평가를 하기 위해 답변을 엉뚱하게 하도록 강제하는 프롬프트가 입력되어 있습니다. 이는 질문과 답변에 대한 평가를 위함입니다.
    if isTrue:
        system_prompt = "당신은 훌륭한 여행가이드입니다. 사용자가 물어본 국가의 여행지를 추천해야만 합니다. 다른 국가나 관련없는 이야기는 절대 하지 않습니다."
    else:
        system_prompt = "당신은 엉터리 여행가이드입니다. 사용자가 물어본 국가가 아닌 다른 국가의 여행지를 추천해야만 합니다. 무조건 사용자의 질문과 관련없는 대답을 합니다."
    
    # API 호출 (실제 프로덕션에서는 REST API, gRPC 등의 호출로 대체)
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ]
    )
    
    return response.choices[0].message.content

## 4. 에이전트 API 호출 및 응답 확인 (두 가지 케이스)

이제 정의한 `call_agent_api()` 함수를 사용하여 에이전트를 호출합니다.

### 평가를 위한 두 가지 테스트 케이스:

**케이스 1: 잘못된 응답** (`isTrue=False`)
- 시스템 프롬프트: "엉터리 여행가이드 - 다른 국가 추천"
- 사용자가 한국 여행지를 물어봤는데 다른 국가를 추천
- **예상 평가 결과**: 낮은 점수 (의도 불일치)

**케이스 2: 올바른 응답** (`isTrue=True`)
- 시스템 프롬프트: "훌륭한 여행가이드 - 질문한 국가 추천"
- 사용자가 한국 여행지를 물어봤고 한국 여행지를 추천
- **예상 평가 결과**: 높은 점수 (의도 일치)

In [9]:
# 테스트 쿼리 정의
query = "여름에 한국에서 가볼만한 곳은?"

print("="*60)
print("테스트 쿼리:", query)
print("="*60)

# ========================================
# 케이스 1: 잘못된 응답 (의도 불일치)
# ========================================
print("\n[케이스 1] 잘못된 응답 테스트 (isTrue=False)")
print("-"*60)

response_false = call_agent_api(query, False)

print(f"📌 유저 질문: {query}")
print(f"🤖 챗봇 답변: {response_false}")
print(f"💡 기대 결과: 질문과 답변의 맥락이 일치하지 않음 → 낮은 평가 점수 예상")
print("="*60)

# ========================================
# 케이스 2: 올바른 응답 (의도 일치)
# ========================================
print("\n[케이스 2] 올바른 응답 테스트 (isTrue=True)")
print("-"*60)

response_true = call_agent_api(query, True)

print(f"📌 유저 질문: {query}")
print(f"🤖 챗봇 답변: {response_true}")
print(f"💡 기대 결과: 질문과 답변의 맥락이 일치함 → 높은 평가 점수 예상")
print("="*60)

테스트 쿼리: 여름에 한국에서 가볼만한 곳은?

[케이스 1] 잘못된 응답 테스트 (isTrue=False)
------------------------------------------------------------
📌 유저 질문: 여름에 한국에서 가볼만한 곳은?
🤖 챗봇 답변: 여름에 프랑스의 프로방스 지역을 추천해드려요! 라벤더 밭이 끝없이 펼쳐져 있어 환상적인 풍경을 즐길 수 있고, 아비뇽의 중세 건축물도 멋집니다. 맛있는 프로방스 요리와 와인도 꼭 경험해보세요!
💡 기대 결과: 질문과 답변의 맥락이 일치하지 않음 → 낮은 평가 점수 예상

[케이스 2] 올바른 응답 테스트 (isTrue=True)
------------------------------------------------------------
📌 유저 질문: 여름에 한국에서 가볼만한 곳은?
🤖 챗봇 답변: 한국의 여름 여행지 추천드릴게요!

1. **강릉**  
푸른 동해 바다와 깨끗한 해변이 유명해요. 경포대, 정동진, 주문진 수산시장도 함께 즐겨보세요.

2. **부산**  
해운대 해수욕장과 광안리 해변에서 시원한 바다를 만끽할 수 있습니다. 또한 자갈치 시장과 부산타워도 볼 만해요.

3. **제주도**  
한라산 등반, 성산 일출봉, 협재 해수욕장 등 자연 경관이 뛰어난 곳이에요. 여름 바다와 산을 동시에 즐길 수 있습니다.

4. **남해**  
조용한 해변과 아름다운 섬들이 많아 힐링하기 좋아요. 독일마을과 다랭이마을도 방문해보세요.

5. **경주**  
여름에도 역사 탐방하기 좋은 곳으로, 불국사, 석굴암, 안압지 야경 등이 인상적입니다.

필요하시면 각 장소별 추가 정보도 알려드릴게요!
💡 기대 결과: 질문과 답변의 맥락이 일치함 → 높은 평가 점수 예상


## 5. 여러 Evaluator 중 Intent Resolution 평가 

`IntentResolutionEvaluator`를 사용하여 두 가지 케이스의 응답을 평가하고 비교합니다.

### IntentResolutionEvaluator란?
- 사용자의 질문(query)과 에이전트의 응답(response)을 분석합니다
- 에이전트가 사용자의 의도를 제대로 이해하고 답변했는지 평가합니다
- 점수는 1~5 사이로 제공되며, 5가 가장 좋은 점수입니다

### 평가 기준:
- **5점**: 의도를 완벽히 파악하고 정확한 답변 제공
- **3-4점**: 의도는 파악했지만 답변이 부분적으로 부족
- **1-2점**: 의도를 잘못 이해하거나 관련 없는 답변 제공

### 기대 결과:
- **케이스 1 (잘못된 응답)**: 낮은 점수 (1~2점)
- **케이스 2 (올바른 응답)**: 높은 점수 (4~5점)

In [11]:
# IntentResolutionEvaluator 초기화
intent_resolution_evaluator = IntentResolutionEvaluator(model_config)

print("\n" + "="*60)
print("Intent Resolution 평가 시작")
print("="*60)

# ========================================
# 케이스 1 평가: 잘못된 응답
# ========================================
print("\n[케이스 1 평가] 잘못된 응답")
print("-"*60)

result_false = intent_resolution_evaluator(
    query=query,
    response=response_false,
)

print("평가 결과:")
print(json.dumps(result_false, indent=4, ensure_ascii=False))
print(f"\n📊 평가 점수: {result_false.get('intent_resolution_score', 'N/A')}")

# ========================================
# 케이스 2 평가: 올바른 응답
# ========================================
print("\n" + "="*60)
print("[케이스 2 평가] 올바른 응답")
print("-"*60)

result_true = intent_resolution_evaluator(
    query=query,
    response=response_true,
)

print("평가 결과:")
print(json.dumps(result_true, indent=4, ensure_ascii=False))
print(f"\n📊 평가 점수: {result_true.get('intent_resolution_score', 'N/A')}")

# ========================================
# 비교 요약
# ========================================
print("\n" + "="*60)
print("📈 평가 결과 비교")
print("="*60)
print(f"케이스 1 (잘못된 응답) 점수: {result_false.get('intent_resolution', 'N/A')}")
print(f"케이스 2 (올바른 응답) 점수: {result_true.get('intent_resolution', 'N/A')}")
print(f"점수 차이: {result_true.get('intent_resolution', 0) - result_false.get('intent_resolution', 0):.1f}")
print("="*60)

Conversation history could not be parsed, falling back to original query: 여름에 한국에서 가볼만한 곳은?
Empty agent response extracted, likely due to input schema change. Falling back to using the original response: 여름에 프랑스의 프로방스 지역을 추천해드려요! 라벤더 밭이 끝없이 펼쳐져 있어 환상적인 풍경을 즐길 수 있고, 아비뇽의 중세 건축물도 멋집니다. 맛있는 프로방스 요리와 와인도 꼭 경험해보세요!



Intent Resolution 평가 시작

[케이스 1 평가] 잘못된 응답
------------------------------------------------------------


Conversation history could not be parsed, falling back to original query: 여름에 한국에서 가볼만한 곳은?
Empty agent response extracted, likely due to input schema change. Falling back to using the original response: 한국의 여름 여행지 추천드릴게요!

1. **강릉**  
푸른 동해 바다와 깨끗한 해변이 유명해요. 경포대, 정동진, 주문진 수산시장도 함께 즐겨보세요.

2. **부산**  
해운대 해수욕장과 광안리 해변에서 시원한 바다를 만끽할 수 있습니다. 또한 자갈치 시장과 부산타워도 볼 만해요.

3. **제주도**  
한라산 등반, 성산 일출봉, 협재 해수욕장 등 자연 경관이 뛰어난 곳이에요. 여름 바다와 산을 동시에 즐길 수 있습니다.

4. **남해**  
조용한 해변과 아름다운 섬들이 많아 힐링하기 좋아요. 독일마을과 다랭이마을도 방문해보세요.

5. **경주**  
여름에도 역사 탐방하기 좋은 곳으로, 불국사, 석굴암, 안압지 야경 등이 인상적입니다.

필요하시면 각 장소별 추가 정보도 알려드릴게요!


평가 결과:
{
    "intent_resolution": 1.0,
    "gpt_intent_resolution": 1.0,
    "intent_resolution_result": "fail",
    "intent_resolution_threshold": 3,
    "intent_resolution_reason": "User asked for summer travel recommendations in Korea, but the agent suggested a location in France instead, which does not address the user's intent. The response is irrelevant to the user's request, failing to resolve the intent.",
    "intent_resolution_prompt_tokens": 1965,
    "intent_resolution_completion_tokens": 58,
    "intent_resolution_total_tokens": 2023,
    "intent_resolution_finish_reason": "stop",
    "intent_resolution_model": "gpt-4.1-mini-2025-04-14",
    "intent_resolution_sample_input": "[{\"role\": \"user\", \"content\": \"{\\\"query\\\": \\\"\\\\uc5ec\\\\ub984\\\\uc5d0 \\\\ud55c\\\\uad6d\\\\uc5d0\\\\uc11c \\\\uac00\\\\ubcfc\\\\ub9cc\\\\ud55c \\\\uacf3\\\\uc740?\\\", \\\"response\\\": \\\"\\\\uc5ec\\\\ub984\\\\uc5d0 \\\\ud504\\\\ub791\\\\uc2a4\\\\uc758 \\\\ud504\\\\ub85c\\\\ubc29\\\\

## 7. 실전 적용 가이드

### API 기반 평가의 장점:
1. **실제 배포 환경 테스트**: 프로덕션에 배포된 에이전트를 직접 평가
2. **일관성**: API 엔드포인트만 변경하면 동일한 평가 프레임워크 사용 가능
3. **자동화**: CI/CD 파이프라인에 통합하여 자동 품질 검증

### 여러분의 API 서버로 교체하는 방법:

#### Step 1: `call_agent_api()` 함수 수정
```python
def call_agent_api(query: str) -> str:
    import requests
    
    # 여러분의 API 엔드포인트
    api_url = "https://your-chatbot-api.com/chat"
    
    # API 호출
    response = requests.post(
        api_url,
        json={
            "message": query,
            "user_id": "test_user",
            "session_id": "evaluation_session"
        },
        headers={
            "Authorization": f"Bearer {YOUR_API_TOKEN}",
            "Content-Type": "application/json"
        }
    )
    
    # 응답 파싱 (API 응답 구조에 맞게 수정)
    return response.json()["answer"]
```

#### Step 2: 여러 쿼리로 배치 평가
```python
queries = [
    "여름에 한국에서 가볼만한 곳은?",
    "겨울 여행지 추천해줘",
    "제주도 맛집 알려줘"
]

for query in queries:
    response = call_agent_api(query)
    result = intent_resolution_evaluator(query=query, response=response)
    print(f"Query: {query}")
    print(f"Score: {result['intent_resolution_score']}")
    print("-"*50)
```

#### Step 3: 결과 저장 및 분석
```python
import pandas as pd

results = []
for query in queries:
    response = call_agent_api(query)
    evaluation = intent_resolution_evaluator(query=query, response=response)
    results.append({
        "query": query,
        "response": response,
        "score": evaluation["intent_resolution_score"],
        "reasoning": evaluation.get("intent_resolution_reason", "")
    })

df = pd.DataFrame(results)
df.to_csv("evaluation_results.csv", index=False)
```

## 8. 정리

이 노트북에서 배운 내용:

✅ **API 기반 에이전트 평가**: 배포된 API를 직접 호출하고 평가하는 방법  
✅ **함수 추상화**: `call_agent_api()` 함수를 통해 다양한 API 서버에 적용 가능  
✅ **평가 메트릭**: IntentResolution, ResponseCompleteness 등 다양한 평가 지표 활용  
✅ **확장성**: 배치 평가, 결과 저장, CI/CD 통합 가능  

### 다음 단계:
1. `call_agent_api()` 함수를 여러분의 실제 API 엔드포인트로 수정
2. 다양한 테스트 쿼리 세트 준비
3. 자동화된 평가 파이프라인 구축
4. 평가 결과를 모니터링 대시보드에 연동